In [39]:
from bs4 import BeautifulSoup
import urllib2
import csv
from os import listdir
from os.path import isfile, join
import pandas as pd

BASE_URL = "http://androidweekly.net/issues/issue-{}"

In [25]:
def contentForIssue(num):
    url = BASE_URL.format(num)
    print url
    request = urllib2.Request(url)
    request.add_header('Accept-Encoding', 'utf-8')
    response = urllib2.urlopen(url)
    soup = BeautifulSoup(response.read().decode('utf-8'), 'lxml')
    content = soup.find('div', { 'class':'issue' })
    return content

In [26]:
def dataForIssue(content, issue_num):
    sections = content.find_all('td')
    last_section_header = ""
    data = []
    section_count = 0
    for section in sections:
        if section.h2 or section.h3:
            section_count += 1
            if section.h2:
                last_section_header = section.h2.text
            else:
                last_section_header = section.h3.text
        if section.a and section.a.string:
            item = {}
            item['issue_num'] = issue_num
            item['section'] = last_section_header.encode("utf-8").strip()
            item['title'] = section.a.text.encode("utf-8").replace('\n','').strip()
            item['link'] = section.a.get('href').encode("utf-8").strip()
            item['site'] = section.span.text.replace('(','').replace(')', '').replace('\n','').encode("utf-8").strip()
            item['description'] = section.p.text.replace('\n','').encode("utf-8").strip()
            data.append(item)
    print 'Found %d items in %s sections' % (len(data), section_count)
    return data

In [27]:
def dataForIssueBefore103(content, issue_num):
    sections = content.find_all(['h2', 'div', 'p'])
    last_section_header = ""
    data = []
    section_count = 0
    for section in sections:
        if section.name == 'h2':
            section_count += 1
            last_section_header = section.text
        elif section.name == 'div' and len(section.find_all('a')) > 1:
            links = section.find_all('a');
            item = {}
            item['issue_num'] = issue_num
            item['section'] = last_section_header.encode("utf-8").strip()
            item['title'] = links[1].string.encode("utf-8").replace('\n','').strip()
            item['link'] = links[1].get('href').encode("utf-8").strip()
            item['site'] = section.span.text.replace('(','').replace(')', '').replace('\n','').encode("utf-8").strip()
            item['description'] = section.p.text.replace('\n','').encode("utf-8").strip()
            data.append(item)
        elif section.name == 'p' and section.a and section.a.text:
            item = {}
            item['issue_num'] = issue_num
            item['section'] = last_section_header.encode("utf-8").strip()
            item['title'] = section.a.string.encode("utf-8").replace('\n','').strip()
            item['link'] = section.a.get('href').encode("utf-8").strip()
            if section.span:
                item['site'] = section.span.text.replace('(','').replace(')', '').replace('\n','').encode("utf-8").strip()
            else:
                item['site'] = ""
            if section.br and section.br.next_sibling:
                item['description'] = section.br.next_sibling.replace('\n','').encode("utf-8").strip()
            else:
                item['description'] = ""
            data.append(item)
    print 'Found %d items in %s sections' % (len(data), section_count)
    return data

In [28]:
def dataForIssueBefore60(content, issue_num):
    return dataForIssueBefore103(content.div, issue_num)

In [29]:
def writeData(data, fname):
    with open(fname, 'wb') as f:
        w = csv.DictWriter(f, data[0].keys())
        w.writeheader()
        for datum in data:
            w.writerow(datum)

In [30]:
def getAllData(issueNums):
    for i in issueNums:
        content = contentForIssue(i)
        data = None
        if (i < 60):
            data = dataForIssueBefore60(content, i) 
        elif (i < 103):
            data = dataForIssueBefore103(content, i) 
        else:
            data = dataForIssue(content, i)
        writeData(data, 'issue_{}.csv'.format(i))

In [33]:
# 500 errors: 138, 108, 43
# Skipped b/c unusual: 30
issueNums = range(1, 217)
issueNums.remove(138)
issueNums.remove(108)
issueNums.remove(43)
issueNums.remove(30)
getAllData(reversed(issueNums))

http://androidweekly.net/issues/issue-216
Found 26 items in 6 sections
http://androidweekly.net/issues/issue-215
Found 28 items in 7 sections
http://androidweekly.net/issues/issue-214
Found 21 items in 8 sections
http://androidweekly.net/issues/issue-213
Found 30 items in 7 sections
http://androidweekly.net/issues/issue-212
Found 23 items in 7 sections
http://androidweekly.net/issues/issue-211
Found 29 items in 9 sections
http://androidweekly.net/issues/issue-210
Found 28 items in 9 sections
http://androidweekly.net/issues/issue-209
Found 27 items in 10 sections
http://androidweekly.net/issues/issue-208
Found 25 items in 7 sections
http://androidweekly.net/issues/issue-207
Found 25 items in 8 sections
http://androidweekly.net/issues/issue-206
Found 25 items in 8 sections
http://androidweekly.net/issues/issue-205
Found 34 items in 8 sections
http://androidweekly.net/issues/issue-204
Found 34 items in 10 sections
http://androidweekly.net/issues/issue-203
Found 29 items in 7 sections
http

In [54]:
def loadDataFiles():
    files = [f for f in listdir('data') if isfile(join('data', f)) and f.endswith('.csv')]
    df = None
    for fname in files:
        if df is None:
            df = pd.read_csv('data/' + fname)
        else:
            df = df.append(pd.read_csv('data/' + fname), ignore_index=True)
    return df
    
df = loadDataFiles()
df.head()

,description,title,section,site,issue_num,link
0,An Android Tutorial aimed for iOS Developers.,http://clayallsopp.posterous.com/building-an-a...,Articles and Tutorials,NaN,1,http://clayallsopp.posterous.com/building-an-a...
1,Selenium Webdriver is now available for your A...,http://android-developers.blogspot.com/2011/10...,Articles and Tutorials,NaN,1,http://android-developers.blogspot.com/2011/10...
2,For our dev rookies we got an Java tutorial wh...,http://mobile.tutsplus.com/tutorials/android/j...,Articles and Tutorials,NaN,1,http://mobile.tutsplus.com/tutorials/android/j...
3,The brandnew nexus with Android 4.0 aka Ice Cr...,http://www.google.com/nexus/,Headlines,NaN,1,http://www.google.com/nexus/
4,A very cool info graphic about the Rise of And...,http://bbgeeks.com/images/AndroidRise.png,Headlines,NaN,1,http://bbgeeks.com/images/AndroidRise.png


In [58]:
df.groupby('section').count().sort('issue_num', ascending=False).head()

,description,title,site,issue_num,link
section,,,,,
Articles & Tutorials,1543,1558,1557,1558,1558
Libraries & Code,707,709,709,709,709
Jobs,443,459,459,459,459
Sponsored,219,219,219,219,219
News,192,193,193,193,193


In [61]:
adf = df[df.section == 'Articles & Tutorials']
adf.issue_num.count()

1558

In [97]:
adf.groupby('site').count().sort('issue_num', ascending=False).head()

,description,title,section,issue_num,link,medium_author
site,,,,,,
medium.com,176,176,176,176,176,176
blog.stylingandroid.com,155,163,163,163,163,0
plus.google.com,90,90,90,90,90,0
android-developers.blogspot.com,59,59,59,59,59,0
speakerdeck.com,29,29,29,29,29,0


In [104]:
mdf = adf[adf.site == 'medium.com']
mdf.loc[:,('medium_author')] = mdf.link.str.split('/').str.get(3)

In [105]:
mdf.groupby('medium_author').count().sort('issue_num', ascending=False)['issue_num'].head()

medium_author
google-developers           22
ribot-labs                   9
sebs-top-tips                8
@hitherejoe                  6
google-developer-experts     5
Name: issue_num, dtype: int64